In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageOps
import tkinter as tk
from tkinter import filedialog, simpledialog, Scale, messagebox
from scipy.stats import norm
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import matplotlib

matplotlib.use('TkAgg')

# Lattice Boltzmann parameters
nx, ny = 200, 100  # Grid size (width, height)
tau = 0.8  # Relaxation time (higher for stability) - Adjust based on solvent viscosity if needed
omega = 1 / tau  # Collision term
dx = dy = 1
dt = 1
rho0 = 1.0  # Initial density
max_time_steps = 1000

# Velocity set for D2Q9 model
c_sqr = 1/3  # Speed of sound squared
w = np.array([4/9] + [1/9]*4 + [1/36]*4)  # Weights (length 9)
c = np.array([
    [0, 0], [1, 0], [0, 1], [-1, 0], [0, -1],
    [1, 1], [-1, 1], [-1, -1], [1, -1]
])  # Directions (9 rows)

# Global variables for simulation state
f = np.ones((9, nx, ny)) * rho0 / 9
rho = np.sum(f, axis=0)
u = np.zeros((2, nx, ny))
ridges_rotated = np.zeros((nx, ny), dtype=bool)
current_time_step = 0

def gaussian_profile(y, center, sigma):
    """Generates a 1D Gaussian profile."""
    return norm.pdf(y, loc=center, scale=sigma)

def initialize_simulation(rotation_angle_degrees, spray_angle_degrees, tip_to_surface_distance):
    """Initializes the simulation fields and ridge pattern."""
    global f, rho, u, ridges_rotated

    # Create a simple ridge pattern
    ridges = np.zeros((nx, ny))
    for i in range(ny):
        if (i // 10) % 2 == 0:
            ridges[:, i] = 1  # Horizontal ridges

    # Rotate the ridge pattern
    img_ridges = Image.fromarray(ridges.astype(np.uint8))
    rotated_img = img_ridges.rotate(rotation_angle_degrees, expand=False, fillcolor=0) # Keep size
    ridges_rotated[:] = np.array(rotated_img) > 0.5

    # Initialize fields
    f[:] = np.ones((9, nx, ny)) * rho0 / 9
    rho[:] = np.sum(f, axis=0)
    u[:] = 0

    # Set DESI spray parameters
    angle_radians = spray_angle_degrees * np.pi / 180
    ux_spray = 0.02 * np.cos(angle_radians)
    uy_spray = 0.02 * np.sin(angle_radians)

    # Apply Gaussian spray velocity at left boundary
    center_y = ny / 2
    sigma = ny / 60 * (tip_to_surface_distance - 1) + ny / 20  # Calculate sigma based on d1
    y_coords = np.arange(ny)
    profile = gaussian_profile(y_coords, center_y, sigma)
    profile /= np.max(profile) # Normalize the profile to have a maximum of 1

    for y in range(ny):
        u[0, 0, y] = ux_spray # Apply uniform x-velocity
        u[1, 0, y] = uy_spray * profile[y] # Modulate y-velocity with Gaussian profile

def update_flow(time_step):
    """Updates the flow for the given number of time steps."""
    global f, rho, u, current_time_step

    target_step = int(time_step)

    def equilibrium(rho, u):
        """Calculate equilibrium distribution function."""
        cu = np.einsum('ia,axy->ixy', c, u)
        usqr = u[0]**2 + u[1]**2
        feq = np.einsum('i,jk->ijk', w, rho) * (1 + 3*cu + 9/2*cu**2 - 3/2*usqr)
        return feq

    for _ in range(target_step - current_time_step):
        if current_time_step >= max_time_steps:
            break
        # Compute equilibrium
        feq = equilibrium(rho, u)

        # Collision step
        f += omega * (feq - f)

        # Streaming step
        for i, ci in enumerate(c):
            f[i] = np.roll(f[i], ci, axis=(0, 1))

        # Apply outflow boundary condition at the right edge (x = nx - 1)
        for y in range(ny):
            # Particles moving out (positive x-velocity)
            pass # These particles leave the domain naturally

            # Particles moving in (negative x-velocity) - Apply zero gradient
            if c[i][0] == -1:
                f[i, nx - 1, y] = f[i, nx - 2, y]

        # Recompute macroscopic variables
        rho = np.maximum(np.sum(f, axis=0), 1e-6)  # Prevent zero density
        u[0] = np.sum(f * c[:, 0, None, None], axis=0) / rho
        u[1] = np.sum(f * c[:, 1, None, None], axis=0) / rho

        # Enforce bounce-back on ridges (zero velocity)
        u[:, ridges_rotated] = 0  # Corrected bounce-back condition
        current_time_step += 1
    plot_flow()

def plot_flow():
    """Plots the current state of the solvent flow."""
    global fig, canvas, u, ridges_rotated

    plt.clf()
    plt.imshow(u[0].T, cmap='jet', origin='lower', extent=[0, nx, 0, ny])
    plt.colorbar(label='Solvent Flow Velocity')
    plt.xlabel("X-axis (Flow Direction)")
    plt.ylabel("Y-axis (Fingerprint Surface)")
    plt.title(f"DESI Solvent Flow Over Generated Ridge (t={current_time_step})")

    if show_ridges_var.get():
        ridge_y, ridge_x = np.where(ridges_rotated.T) # Transpose for correct indexing
        plt.scatter(ridge_x, ridge_y, color='white', s=1, alpha=0.5, label='Ridges')
        plt.legend()

    canvas.draw()

def start_simulation_from_gui():
    rotation = rotation_angle_scale.get()
    spray_angle = spray_angle_scale.get()
    tip_distance = tip_to_surface_distance_scale.get()
    global current_time_step
    current_time_step = 0
    initialize_simulation(rotation, spray_angle, tip_distance)
    time_step_slider.config(to=max_time_steps)
    update_flow(0) # Show initial state

# --- GUI Setup ---
root = tk.Tk()
root.title("DESI Solvent Flow Simulator")

# --- Schematic Window ---
schematic_window = tk.Toplevel(root)
schematic_window.title("DESI Emitter Schematic")
schematic_canvas_width = 300
schematic_canvas_height = 200
schematic_canvas = tk.Canvas(schematic_window, width=schematic_canvas_width, height=schematic_canvas_height, bg='white')
schematic_canvas.pack()

def update_schematic(spray_angle, tip_distance):
    schematic_canvas.delete("all")  # Clear previous drawing
    surface_y = schematic_canvas_height * 0.8
    surface_height = 20
    surface_x_start = 20
    surface_x_end = schematic_canvas_width - 20
    schematic_canvas.create_rectangle(surface_x_start, surface_y, surface_x_end, surface_y + surface_height, fill='lightgray', outline='black')
    schematic_canvas.create_text((surface_x_start + surface_x_end) / 2, surface_y + surface_height + 10, text='Surface', anchor=tk.CENTER)

    spray_angle_rad = np.radians(spray_angle)
    sprayer_length = 80
    tip_y = surface_y - tip_distance * 10  # Scale tip distance for visualization
    tip_x = 50

    end_x = tip_x + sprayer_length * np.cos(spray_angle_rad)
    end_y = tip_y - sprayer_length * np.sin(spray_angle_rad)

    schematic_canvas.create_line(tip_x, tip_y, end_x, end_y, width=2)
    schematic_canvas.create_oval(tip_x - 5, tip_y - 5, tip_x + 5, tip_y + 5, fill='blue') # Sprayer tip

    # Label angles and distances (simplified)
    schematic_canvas.create_text(tip_x + 30, tip_y - 20, text=f"α = {spray_angle}°", anchor=tk.W)
    schematic_canvas.create_line(tip_x, tip_y, tip_x, surface_y, dash=(4, 4))
    schematic_canvas.create_text(tip_x + 10, (tip_y + surface_y) / 2, text=f"d1 = {tip_distance:.1f} mm", anchor=tk.W)

# Rotation Angle Control
rotation_angle_label = tk.Label(root, text="Ridge Pattern Rotation Angle (°):")
rotation_angle_label.pack()
rotation_angle_scale = Scale(root, from_=-180, to=180, orient=tk.HORIZONTAL, length=300, resolution=1, label="Rotation Angle")
rotation_angle_scale.set(0) # Default rotation angle
rotation_angle_scale.pack()
rotation_angle_scale.config(command=lambda angle: update_schematic(spray_angle_scale.get(), tip_to_surface_distance_scale.get()))

# Spray Angle Control
spray_angle_label = tk.Label(root, text="DESI Sprayer Angle (°):")
spray_angle_label.pack()
spray_angle_scale = Scale(root, from_=1, to=179, orient=tk.HORIZONTAL, length=300, resolution=1, label="Spray Angle")
spray_angle_scale.set(72) # Default spray angle
spray_angle_scale.pack()
spray_angle_scale.config(command=lambda angle: update_schematic(spray_angle_scale.get(), tip_to_surface_distance_scale.get()))

# Tip to Surface Distance (d1) Control
tip_to_surface_distance_label = tk.Label(root, text="Tip to Surface Distance (d1, mm):")
tip_to_surface_distance_label.pack()
tip_to_surface_distance_scale = Scale(root, from_=1, to=10, orient=tk.HORIZONTAL, length=300, resolution=0.1, label="Tip Distance (d1)")
tip_to_surface_distance_scale.set(5) # Default tip distance
tip_to_surface_distance_scale.pack()
tip_to_surface_distance_scale.config(command=lambda distance: update_schematic(spray_angle_scale.get(), tip_to_surface_distance_scale.get()))

# Toggle for Showing Ridges
show_ridges_var = tk.BooleanVar()
show_ridges_checkbox = tk.Checkbutton(root, text="Show Ridges", variable=show_ridges_var)
show_ridges_checkbox.pack(pady=10)
show_ridges_var.set(True) # Default to showing ridges

# Time Step Control
time_step_label = tk.Label(root, text="Flow Time Step:")
time_step_label.pack()
time_step_slider = Scale(root, from_=0, to=max_time_steps, orient=tk.HORIZONTAL, length=300, resolution=1, label="Time Step", command=update_flow)
time_step_slider.set(0)
time_step_slider.pack()

# Start Simulation Button
start_button = tk.Button(root, text="Initialize Simulation", command=start_simulation_from_gui)
start_button.pack(pady=20)

# --- Matplotlib Plot in Tkinter ---
fig, ax = plt.subplots(figsize=(8, 4))
canvas = FigureCanvasTkAgg(fig, master=root)
canvas_widget = canvas.get_tk_widget()
canvas_widget.pack(pady=10)

# Initial drawing of the schematic
update_schematic(spray_angle_scale.get(), tip_to_surface_distance_scale.get())

root.mainloop()